In [19]:
import os
import sys
import torch
import random
import argparse
import numpy as np
import scipy

sys.path.append('../../timeseries/')
sys.path.append('../../src/')

import factors
import state_models
import observation_models
from ssm_em import StateSpaceEM
from nonlinear_ssm import NonLinearStateSpace_EM

from scipy.stats import norm
from scipy.stats import zscore
from ssm_em import StateSpaceEM
from sklearn.covariance import EmpiricalCovariance
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.gaussian_process.kernels import ExpSineSquared, RBF

import darts
from darts.models import TCNModel
from darts.models import GaussianProcessFilter
from darts.utils.likelihood_models import GaussianLikelihoodModel
from darts.timeseries import TimeSeries
from exp_utils import *


In [18]:
#np.log1p(np.exp(1.00576471e+03))
np.log1p(np.exp(1.00576471e+03))

/tmp/ipykernel_3352/234746507.py:2: RuntimeWarning: overflow encountered in exp
  np.log1p(np.exp(1.00576471e+03))


inf

In [77]:
a = np.asarray([1, 2, 4])
print(a.shape)
a[a<3] = 1
a

(3,)


array([1, 1, 4])

In [78]:
y = np.empty(a.shape)
print(a)
y[a > 10] = a[a > 10]
#y[a <= 10] =
np.log(np.exp(np.amax([a[a <= 10], 1e-5]))- 1)
#np.log(np.exp(np.amax([x[x <= 10], 1e-5])) - 1)


[1 1 4]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [81]:
a[a <= 10]
y[a <= 10]
np.amax(a[a <= 10], 1e-5)

TypeError: 'float' object cannot be interpreted as an integer

In [105]:
a.shape

(3,)

In [104]:
np.amax(np.stack([a[a <= 10], np.repeat(1e-15, 3)]), 0)

array([1., 1., 4.])

In [108]:
np.amax(np.stack([a[a <= 10], np.repeat(1e-15, 3)]).T, 1)#.shape

array([1., 1., 4.])

In [28]:
df = pd.read_csv(filepath_or_buffer='../../data/opsd/time_series_60min_singleindex_filtered.csv')
nar = np.asarray(df)
nar

,utc_timestamp,cet_cest_timestamp,DE_load_actual_entsoe_transparency,DE_solar_generation_actual,DE_wind_generation_actual
0,2014-12-31T23:00:00Z,2015-01-01T00:00:00+0100,NaN,NaN,NaN
1,2015-01-01T00:00:00Z,2015-01-01T01:00:00+0100,41151.0,NaN,8852.0
2,2015-01-01T01:00:00Z,2015-01-01T02:00:00+0100,40135.0,NaN,9054.0
3,2015-01-01T02:00:00Z,2015-01-01T03:00:00+0100,39106.0,NaN,9070.0
4,2015-01-01T03:00:00Z,2015-01-01T04:00:00+0100,38765.0,NaN,9163.0
...,...,...,...,...,...
43843,2020-01-01T18:00:00Z,2020-01-01T19:00:00+0100,50732.0,0.0,13433.0
43844,2020-01-01T19:00:00Z,2020-01-01T20:00:00+0100,48591.0,0.0,13881.0
43845,2020-01-01T20:00:00Z,2020-01-01T21:00:00+0100,47664.0,0.0,14007.0
43846,2020-01-01T21:00:00Z,2020-01-01T22:00:00+0100,46994.0,0.0,12695.0


In [45]:
#np.log(np.exp(nar[:,2]) - 1 +1e-10)
#1.00576471e+03
np.exp(8852)

/tmp/ipykernel_3352/1749063802.py:3: RuntimeWarning: overflow encountered in exp
  np.exp(8852)


inf

In [56]:
np.log10(1e10 - 1)#np.log(1e29)

9.99999999995657

In [2]:
x_tr, x_va, x_te, x_te_na, s_tr_x = load_synthetic_e2(ts=1, train_ratio=0.5)

In [3]:
tcnmodel = TCNModel(input_chunk_length=3,
                                    output_chunk_length=1,
                                    kernel_size=2,
                                    num_filters=4,
                                    dilation_base=2,
                                    dropout=0,
                                    random_state=1,
                                    n_epochs=10,
                                    likelihood=GaussianLikelihoodModel())
            
tcnmodel.fit(x_tr)

[2021-09-09 13:17:53,901] INFO | darts.models.torch_forecasting_model | Train dataset contains 3561 samples.
[2021-09-09 13:17:53,901] INFO | darts.models.torch_forecasting_model | Train dataset contains 3561 samples.
[2021-09-09 13:17:53,916] INFO | darts.models.tcn_model | Number of layers chosen: 1
[2021-09-09 13:17:53,916] INFO | darts.models.tcn_model | Number of layers chosen: 1


KeyboardInterrupt: 

In [4]:
x_tr, x_va, x_te_full, x_te_na, s_tr_x = load_synthetic_e2(ts=0, train_ratio=0.5)
x_tr

array([[ 0.92784997, -0.79513451,  0.58068433, ...,  1.06887109,
         1.32713833, -0.07623804],
       [ 0.7533172 ,  0.04639976,  1.15353304, ...,  0.14460459,
         0.74790419,  0.13855092],
       [ 1.10966754, -0.64909666,  0.7840384 , ...,  1.19348068,
         1.38084113, -0.24407528],
       ...,
       [-0.60601622,  1.04995709,  0.23240672, ..., -0.79904569,
        -0.40357452,  0.55100975],
       [-0.29025918,  0.83898822,  0.08835726, ..., -0.0577966 ,
        -0.29828165, -0.16559227],
       [-0.7881369 ,  0.45833488, -0.04319107, ..., -0.50395827,
        -0.03368635,  1.19174428]])

In [5]:
Dz = 1
Dx = x_tr.shape[1]
sm = state_models.LinearStateModel(Dz)
om = observation_models.LinearObservationModel(Dx, Dz)
ssm_em_lin = StateSpaceEM(x_tr, observation_model=om, state_model=sm)
ssm_em_lin.run()

Iteration 2 - llk=-11579.0
Iteration 4 - llk=-9446.2
Iteration 6 - llk=-7054.6
Iteration 8 - llk=-4963.3
Iteration 10 - llk=-3588.0
Iteration 12 - llk=-3068.1
Iteration 14 - llk=-2938.4
Iteration 16 - llk=-2887.9
Iteration 18 - llk=-2856.6
Iteration 20 - llk=-2834.1
Iteration 22 - llk=-2816.1
Iteration 24 - llk=-2800.4
Iteration 26 - llk=-2785.7
Iteration 28 - llk=-2771.5
Iteration 30 - llk=-2757.5
Iteration 32 - llk=-2743.7
Iteration 34 - llk=-2730.0
Iteration 36 - llk=-2716.3
Iteration 38 - llk=-2702.7
Iteration 40 - llk=-2689.2
Iteration 42 - llk=-2675.8
Iteration 44 - llk=-2662.4
Iteration 46 - llk=-2649.1
Iteration 48 - llk=-2635.8
Iteration 50 - llk=-2622.6
Iteration 52 - llk=-2609.6
Iteration 54 - llk=-2596.5
Iteration 56 - llk=-2583.6
Iteration 58 - llk=-2570.8
Iteration 60 - llk=-2558.0
Iteration 62 - llk=-2545.3
Iteration 64 - llk=-2532.7
Iteration 66 - llk=-2520.2
Iteration 68 - llk=-2507.7
Iteration 70 - llk=-2495.4
Iteration 72 - llk=-2483.2
Iteration 74 - llk=-2471.0
Iter